In [12]:
import asyncio
from copra.websocket import Channel, Client
import matplotlib.pyplot as plt
from collections import OrderedDict
from time import sleep
from dateutil import parser
import copy
import datetime
import itertools
from operator import itemgetter
import numpy as np
from dateutil.tz import tzutc
import math
import pytz
from pytz import timezone
import pickle
import pylab
from scipy.stats import probplot,expon,kstest
import matplotlib.pyplot as plt

In [21]:
def parse_file(file_name):
    raw_updates = []
    with open(file_name,"rb") as f:
        starting_time,starting_bids,starting_asks = list(pickle.load(f))
        while True:
            try:
                raw_updates.append(pickle.load(f))
            except EOFError:
                break
    
    Bids = {}
    Asks = {}    
    
    updates = []
    
    for price, amount in starting_bids:
        Bids[int(round((float(price)*100)))] = float(amount)
    for price, amount in starting_asks:
        Asks[int(round((float(price)*100)))] = float(amount)
        
    for u in raw_updates:
        price = int(round((float(u["price"])*100)))  
        side = u["side"]
        amount = float(u["amount"])
        time = parser.parse(u["time"])
        
        if side == "buy":
            change = amount - Bids.get(price,0)
        else:
            change = amount - Asks.get(price,0)
        
        updates.append({\
            "Bids": copy.copy(Bids), \
            "Asks": copy.copy(Asks), \
            "time": time, \
            "side": side, \
            "price": price, \
            "change": change            
        })
        
        if side == "buy":
            if amount == "0":
                del Bids[price]
            else:
                Bids[price] = float(amount)
        else:
            if amount == "0":
                del Asks[price]
            else:
                Asks[price] = float(amount)   

    
    return(starting_time, updates)

parsed_file = parse_file("12_25_18_data.pkl")
len(parsed_file[1])

28870

In [22]:
def shortened_updates(file_name,K):
    starting_time, updates = parse_file(file_name)
    
    res = []
    
    # Caluclate first reference price
    starting_bids = updates[0]['Bids']
    starting_asks = updates[0]['Asks']
    sorted_bids = list(reversed(sorted(starting_bids.items())))
    sorted_asks = list(sorted(starting_asks.items()))        
    best_bid = sorted_bids[0][0]
    best_ask = sorted_asks[0][0]
    if ((best_bid + best_ask) % 2) != 0:
        old_reference_price = round((best_bid+best_ask)/2, 1)
    else:
        old_reference_price = round((best_bid+best_ask+1)/2, 1)
            
    for u in updates:      
        # Find reference price
        sorted_bids = list(reversed(sorted(u['Bids'].items())))
        sorted_asks = list(sorted(u['Asks'].items()))        
        best_bid = sorted_bids[0][0]
        best_ask = sorted_asks[0][0]
        if ((best_bid + best_ask) % 2) != 0:
            reference_price = round((best_bid+best_ask)/2, 1)
        else:
            middle = (best_bid+best_ask)/2
            if old_reference_price > middle:
                reference_price = round((best_bid+best_ask)/2 + 0.5,1)
            else:
                old_reference_price = round((best_bid+best_ask)/2 - 0.5,1)
                
        shortened_book = OrderedDict([(k,0) for k in range(-K,K+1) if k != 0])
        first_bid = int(round(reference_price - 0.5))
        first_ask = int(round(reference_price + 0.5))        
        for k in range(-K,0):
            shortened_book[k] = u['Bids'].get(first_bid + k + 1,0)
        for k in range(1,K+1):
            shortened_book[k] = u['Asks'].get(first_ask + k - 1,0)
            
        # Find k from the price. Keep track of event if
        # abs(k) <= K
        price = u["price"]
        k = price - reference_price
        if k < 0:
            k = int(round(k - 0.5))
        else:
            k = int(round(k + 0.5))
        if abs(k) <= K:                 
            res.append({
                'reference_price': reference_price,
                'LOB': copy.copy(shortened_book),
                'k': k,
                'change': u['change'],
                'time': u['time']
            })
        
        old_reference_price = reference_price

    return starting_time, res

starting_time,updates = shortened_updates('12_25_18_data.pkl',10)
len(updates)

27922

In [23]:
grouped_by_time = OrderedDict([(k, list(v)) for k, v in itertools.groupby(updates, key=lambda x:x['time'])])
# Contains dictionary of time, reference price, order book, list of changes
cleaned_updates = []
for t, us in grouped_by_time.items():
    if len(us) == 1:
        u = us[0]
        cleaned_updates.append(copy.copy(u))
    else:
        new_update = {'time': t}
        us = sorted(us, key=lambda u:-abs(u['k']))
        grouped_by_k = OrderedDict((k, list(v)) for k, v in itertools.groupby(us, key=lambda u:u['k']))
        reference_k = list(grouped_by_k.keys())[0]
        new_update['reference_price'] = grouped_by_k[reference_k][0]['reference_price']
        new_update['LOB'] = copy.copy(grouped_by_k[reference_k][0]['LOB'])
        events = []
        for k in grouped_by_k:
            combined_change = 0
            for u in grouped_by_k[k]:
                combined_change = combined_change + u['change']
            events.append((k,combined_change))
        for k,change in events:
            new_update = copy.deepcopy(new_update)
            new_update['k'] = k
            new_update['change'] = change
            cleaned_updates.append(new_update)

cleaned_updates
len(cleaned_updates)

27720

In [24]:
combined_updates = []
i = 0
while i < len(cleaned_updates):
    reference_price = cleaned_updates[i]['reference_price']
    j = i
    updates_at_reference = []
    while (j < len(cleaned_updates)) and (cleaned_updates[j]['reference_price'] == reference_price):
        updates_at_reference.append(cleaned_updates[j])
        j += 1
    updates_at_reference = sorted(updates_at_reference,key=lambda u:u['k'])
    grouped_by_k = OrderedDict((k, list(v)) for k, v in itertools.groupby(updates_at_reference, key=lambda u:u['k']))
    for k,us in grouped_by_k.items():
        us = copy.deepcopy(sorted(us,key=lambda u:u['time']))
        keep_index = [True for u in us]
        for m in reversed(range(1,len(us))):
            quick_same_order = (us[m]['time'] - us[m-1]['time']).total_seconds() < 0.01
            same_sign = (us[m]['change'] * us[m-1]['change']) > 0
            if quick_same_order and same_sign:
                us[m-1]['change'] += us[m]['change']
                keep_index[m] = False
        for (u,keep) in zip(us, keep_index):
            if keep:
                combined_updates.append(u)
    i = j + 1
    
combined_updates = sorted(combined_updates,key=lambda u:u['time'])
len(combined_updates)

26870

In [142]:
from datetime import timedelta
n = 1500
ending_time = updates[-1]['time'] + timedelta(seconds=1)
ending_time + timedelta(seconds = 1)
time_period_length = (ending_time - starting_time).total_seconds()

times = [starting_time + timedelta(seconds=i*time_period_length/n) for i in range(1,n+1)]

changes_at_each_k = OrderedDict([(k,[0 for _ in range(n)]) for k in range(-K,K+1) if k != 0])

i = 0
for u in combined_updates:
    if u['change'] < 0:
        changes_at_each_k[u['k']][i] -= 1
    else:
        changes_at_each_k[u['k']][i] += 1
        
    if u['time'] > times[i]:
        i += 1

In [143]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.DataFrame(changes_at_each_k)
correlations = df.corr().astype('object')

In [144]:
from scipy.stats.stats import pearsonr
significances = correlations.copy()
for k1 in changes_at_each_k:
    for k2 in changes_at_each_k:
        res = pearsonr(changes_at_each_k[k1],changes_at_each_k[k2])
        significances[k1][k2] = res

In [146]:
threshold = 0.10
def color_significant_red(val):
    global threshold
    color = 'red' if abs(val[0]) > threshold else 'black'
    return 'color: %s' % color

colored_table = significances.style.applymap(color_significant_red)
colored_table = colored_table.format(lambda x: "{0:.4f},{1:.4f}".format(x[0],x[1]))
colored_table

,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10
-10,"1.0000,0.0000","0.0000,1.0000","-0.0942,0.0003","-0.0175,0.4976","-0.0561,0.0298","0.0109,0.6729","-0.0233,0.3676","-0.0320,0.2155","-0.0284,0.2709","0.0568,0.0277","-0.0056,0.8285","0.0114,0.6589","-0.0058,0.8235","0.0060,0.8178","-0.0327,0.2060","0.0290,0.2622","-0.0503,0.0514","-0.0131,0.6129","0.0155,0.5482","-0.0320,0.2160"
-9,"0.0000,1.0000","1.0000,0.0000","-0.0781,0.0025","-0.1017,0.0001","-0.0233,0.3680","-0.0724,0.0050","-0.0386,0.1351","0.0212,0.4114","0.0354,0.1709","0.0052,0.8394","0.0325,0.2086","0.0142,0.5829","-0.0478,0.0642","-0.0296,0.2517","-0.0054,0.8339","0.0060,0.8163","0.0501,0.0526","-0.0000,1.0000","0.0000,1.0000","-0.0133,0.6081"
-8,"-0.0942,0.0003","-0.0781,0.0025","1.0000,0.0000","-0.0363,0.1599","-0.1235,0.0000","-0.1017,0.0001","-0.0040,0.8764","0.0365,0.1581","0.0074,0.7756","0.0523,0.0427","0.0145,0.5748","-0.0325,0.2084","0.0090,0.7288","0.0000,1.0000","0.0135,0.6004","-0.0075,0.7716","0.0156,0.5451","0.0339,0.1900","0.0080,0.7558","-0.0083,0.7487"
-7,"-0.0175,0.4976","-0.1017,0.0001","-0.0363,0.1599","1.0000,0.0000","-0.0379,0.1427","-0.1471,0.0000","0.0000,0.9989","-0.0912,0.0004","0.0494,0.0558","0.0438,0.0897","0.0087,0.7350","-0.0660,0.0106","-0.0110,0.6690","-0.0138,0.5937","-0.0050,0.8474","0.0083,0.7475","-0.0232,0.3684","0.0000,0.9996","-0.0178,0.4903","0.0000,0.9985"
-6,"-0.0561,0.0298","-0.0233,0.3680","-0.1235,0.0000","-0.0379,0.1427","1.0000,0.0000","-0.0639,0.0133","-0.1963,0.0000","-0.0631,0.0144","0.0132,0.6099","0.0487,0.0592","-0.0138,0.5940","-0.0299,0.2468","0.0143,0.5811","0.0018,0.9435","0.0020,0.9370","-0.0045,0.8620","-0.0310,0.2296","0.0040,0.8758","-0.0047,0.8545","0.0247,0.3395"
-5,"0.0109,0.6729","-0.0724,0.0050","-0.1017,0.0001","-0.1471,0.0000","-0.0639,0.0133","1.0000,0.0000","0.0669,0.0096","-0.1558,0.0000","-0.0685,0.0080","-0.0183,0.4799","-0.0113,0.6613","0.0191,0.4599","0.0343,0.1847","-0.0342,0.1854","0.0201,0.4373","-0.0275,0.2866","-0.0243,0.3463","-0.0001,0.9979","-0.0155,0.5488","0.0113,0.6629"
-4,"-0.0233,0.3676","-0.0386,0.1351","-0.0040,0.8764","0.0000,0.9989","-0.1963,0.0000","0.0669,0.0096","1.0000,0.0000","-0.0909,0.0004","-0.0875,0.0007","0.0388,0.1333","-0.0222,0.3901","0.0097,0.7071","0.0038,0.8829","0.0102,0.6933","0.0121,0.6384","0.0001,0.9966","-0.0207,0.4234","-0.0067,0.7948","0.0024,0.9257","-0.0110,0.6700"
-3,"-0.0320,0.2155","0.0212,0.4114","0.0365,0.1581","-0.0912,0.0004","-0.0631,0.0144","-0.1558,0.0000","-0.0909,0.0004","1.0000,0.0000","-0.1014,0.0001","-0.0428,0.0978","-0.0844,0.0011","-0.0475,0.0659","0.0247,0.3396","0.0437,0.0907","-0.0400,0.1212","-0.0067,0.7949","-0.0244,0.3459","-0.0111,0.6663","-0.0074,0.7732","-0.0341,0.1866"
-2,"-0.0284,0.2709","0.0354,0.1709","0.0074,0.7756","0.0494,0.0558","0.0132,0.6099","-0.0685,0.0080","-0.0875,0.0007","-0.1014,0.0001","1.0000,0.0000","-0.1601,0.0000","-0.0756,0.0034","-0.0438,0.0902","-0.0416,0.1070","0.0615,0.0172","-0.0012,0.9641","0.0398,0.1236","-0.0174,0.5008","-0.0246,0.3416","-0.0245,0.3423","-0.0051,0.8432"
-1,"0.0568,0.0277","0.0052,0.8394","0.0523,0.0427","0.0438,0.0897","0.0487,0.0592","-0.0183,0.4799","0.0388,0.1333","-0.0428,0.0978","-0.1601,0.0000","1.0000,0.0000","-0.0203,0.4321","-0.1839,0.0000","-0.0585,0.0234","-0.0504,0.0509","0.0417,0.1064","0.0363,0.1605","0.0391,0.1300","-0.0654,0.0113","-0.0195,0.4508","-0.0800,0.0019"
